### Lecture 1, step 3: Using Essentia-streaming for computing the features

In this step we will be using Essentia in the streaming mode for the same task
!!Bug observed, notebook to 

In [ ]:
import os
import sys
import essentia
from essentia.streaming import *
from essentia.standard import YamlOutput, YamlInput
import matplotlib.pyplot as plt

%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
import IPython.display as ipd

#Container for analysis parameters
class AnalysisParams:
    def __init__(self,windowSize,hopSize,windowFunction,fftN,fs):
        '''
        windowSize: milliseconds,
        hopSize: milliseconds,
        windowFunction: str ('blackman','hanning',...)
        fftN: int
        '''
        self.windowSize = windowSize
        self.hopSize = hopSize
        self.windowFunction = windowFunction
        self.fftN=fftN
        self.fs=fs

def computeFeatures(fileData,params):
    '''Computation of the low-level features for a given file
    Parameters
    ----------
    fileData : dict
        Dictionary containing all info and data for the file
    params : instance of AnalysisParams
        Analysis parameters
    Returns
    -------
    pool,aggrPool: essentia.Pool
        Pools containing low level and global features
    '''
    #Reading the wave file
    fs=params.fs

    #Windowing (first converting from msec to number of samples)
    windowSize=round(fs*params.windowSize/1000);windowSize=int(windowSize/2)*2#assuring window size is even
    hopSize=round(fs*params.hopSize/1000);hopSize=int(hopSize/2)*2#assuring hopSize is even

    #Instantiating algorithms to be connected in a chain
    loader = MonoLoader(filename = os.path.join(fileData['path'], fileData['name']))
    frameCutter = FrameCutter(frameSize = windowSize, hopSize = hopSize)
    w = Windowing(type = 'hann')
    spec = Spectrum()
    centroid = Centroid(range=int(fs/2))
    flatness = Flatness()
    energy = Energy()
    zcr = ZeroCrossingRate()

    #Connecting the algorithms
    pool = essentia.Pool()
    loader.audio >> frameCutter.signal
    frameCutter.frame >> w.frame >> spec.frame
    w.frame >> energy.array 
    energy.energy >> (pool, 'NRG')
    w.frame >> zcr.signal 
    zcr.zeroCrossingRate >> (pool, 'zcr')
    spec.spectrum >> centroid.array 
    centroid.centroid>> (pool, 'specCentroid')
    spec.spectrum >> flatness.array 
    flatness.flatness >> (pool, 'specFlatness')
    
    return pool
    
def unitNorm(inArray):
    '''Scaling of input array with its largest amplitude
    Parameters
    ----------
    inArray : numpy.array
        Input array
    Returns
    -------
    outArray: numpy.array
        scaled output array
    '''
    outArray=inArray/(np.max(np.abs(inArray)))
    return outArray

Running feature extraction, saving to file and reading from file

In [ ]:
#Setting analysis parameters
# windowSize: milliseconds,hopSize: milliseconds,windowFunction: str ('blackman','hanning',...),fftN: int
params=AnalysisParams(30,10,'hann',2048,44100)
soundsDir='../data/freesound/'

#Gathering all wave files in a folder(including subfolders)
instFiles=dict()#dictionary containing for each instrument a list that carries dictionaries
for root, dirs, files in os.walk(soundsDir):
    for file in files:
        if file.endswith('.wav'):
            instrument=file.split('_')[0]
            fileData=dict();fileData['name']=file;fileData['path']=root;
            files4instrument=instFiles.get(instrument)
            if files4instrument==None:
                files4instrument=[fileData]
            else:
                files4instrument.append(fileData)
            instFiles[instrument]=files4instrument

#Feature extraction for all files of each instrument and saving to files 
for instrument, filesData in instFiles.items():
    for fileInd in range(len(filesData)):
        #Features computation
        pool=computeFeatures(instFiles[instrument][fileInd],params)
        fileName = os.path.join(instFiles[instrument][fileInd]['path'], instFiles[instrument][fileInd]['name'])
        YamlOutput(filename = fileName.replace('.wav','.llfeats.sig'))(pool)

print('Features extracted and saved in files in the same folder as the wave files')